In [1]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import glob
import numpy as np
import cv2
from PIL import Image
c=0
imgdata=[]
imglabel=[]

In [4]:
for i in glob.glob("/content/drive/My Drive/457/*/"):
  for j in glob.glob(i+'*.tif'):
  #x=cv2.imread(i)
    x=np.asarray(Image.open(j).convert('L').resize((700,1400)))
    x=x.flatten()                 #converting to grey scale
  #x.show()
    imgdata.append(x)
    imglabel.append(int(i[-2]))

In [5]:
imgdata=np.asarray(imgdata)
imgdata

array([[253, 252, 252, ..., 254, 254, 254],
       [252, 252, 252, ..., 249, 250, 250],
       [249, 248, 247, ..., 254, 253, 253],
       ...,
       [245, 246, 246, ..., 254, 254, 254],
       [247, 247, 248, ..., 254, 254, 254],
       [247, 248, 247, ..., 252, 253, 254]], dtype=uint8)

In [6]:
imgdata=imgdata/255

In [7]:
labels=[]
for i in range(len(imglabel)):
  labels.append([imglabel[i]])

In [8]:
train=np.concatenate((imgdata,labels),axis=1)

In [9]:
train[0]

array([0.99215686, 0.98823529, 0.98823529, ..., 0.99607843, 0.99607843,
       7.        ])

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_data,test_data=train_test_split(train,test_size=0.25,random_state=10)

In [12]:
train_img,train_label=train_data[:,:-1],train_data[:,-1]
test_img,test_label=test_data[:,:-1],test_data[:,-1]

In [13]:
test_label

array([4., 7., 5., 7., 5., 5., 5., 7., 7., 5., 4., 4., 5., 4., 5., 4., 7.,
       5., 4., 5., 7., 4., 4., 5., 5., 4., 7., 7., 7., 7., 5., 4., 4., 7.,
       4., 5., 5., 4., 7., 5., 7., 5., 7., 4., 7., 5., 4., 7., 7., 7., 7.,
       7., 7., 7., 7., 4.])

In [14]:
from sklearn.svm import SVC

classifier=SVC(kernel='linear', probability=True, random_state=42)
classifier.fit(train_img, train_label)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [15]:
cl = classifier.predict(test_img)
print(cl)


[4. 7. 5. 7. 5. 5. 5. 7. 7. 5. 4. 4. 5. 4. 5. 4. 7. 5. 4. 5. 7. 4. 4. 5.
 5. 4. 7. 7. 7. 7. 5. 4. 4. 7. 5. 5. 5. 4. 7. 5. 7. 5. 7. 4. 7. 5. 4. 7.
 7. 7. 7. 7. 7. 7. 7. 4.]


In [16]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report#precision_recall_fscore_support
cmt=confusion_matrix(test_label,cl)
acr=accuracy_score(test_label,cl)
#pref=precision_recall_fscore_support(test_label,cl,average='macro')
print(classification_report(test_label,cl))

              precision    recall  f1-score   support

         4.0       1.00      0.94      0.97        16
         5.0       0.94      1.00      0.97        17
         7.0       1.00      1.00      1.00        23

    accuracy                           0.98        56
   macro avg       0.98      0.98      0.98        56
weighted avg       0.98      0.98      0.98        56



In [17]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
model1=model.fit(train_img,train_label)
model1.fit(train_img,train_label)
y=model.predict(test_img)
y1=model1.predict(train_img)

In [18]:
print(model1)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [19]:
print(y1)

[4. 5. 7. 5. 5. 5. 5. 7. 4. 7. 4. 4. 4. 4. 4. 4. 5. 5. 4. 7. 4. 4. 5. 4.
 5. 7. 4. 7. 5. 7. 5. 5. 4. 4. 4. 4. 4. 7. 7. 7. 7. 4. 4. 4. 4. 5. 5. 5.
 4. 4. 5. 7. 5. 5. 5. 7. 5. 5. 7. 5. 7. 7. 4. 7. 4. 5. 7. 7. 4. 5. 5. 4.
 7. 7. 7. 5. 7. 4. 5. 4. 7. 4. 5. 4. 7. 5. 7. 5. 5. 5. 4. 4. 5. 7. 5. 7.
 4. 4. 4. 7. 5. 7. 5. 4. 7. 5. 5. 4. 4. 7. 5. 4. 4. 5. 7. 4. 4. 5. 4. 4.
 4. 5. 7. 7. 7. 7. 7. 7. 5. 5. 4. 7. 5. 5. 5. 5. 4. 7. 5. 4. 4. 4. 4. 7.
 7. 5. 5. 5. 7. 5. 5. 7. 5. 4. 7. 7. 7. 7. 4. 4. 4. 5. 5. 7. 7. 5. 7.]


In [20]:
cm=confusion_matrix(test_label,y)
ac=accuracy_score(test_label,y)
tr_ac=accuracy_score(train_label,y1)
#tes_ac=accuracy_score(test_label,model1.predict(test_img))
#pref=precision_recall_fscore_support(test_label,y,average='macro')
print(classification_report(test_label,y))

              precision    recall  f1-score   support

         4.0       1.00      0.75      0.86        16
         5.0       0.85      1.00      0.92        17
         7.0       0.92      0.96      0.94        23

    accuracy                           0.91        56
   macro avg       0.92      0.90      0.90        56
weighted avg       0.92      0.91      0.91        56



In [21]:
print("Accuracy of the model with SVM classification:",acr) #svm accuracy
print("Train accuracy",tr_ac)

Accuracy of the model with SVM classification: 0.9821428571428571
Train accuracy 1.0


In [22]:
print("Accuracy of the model with Decision trees:",ac) #decisiontree accuracy

Accuracy of the model with Decision trees: 0.9107142857142857


In [23]:
print(cm)#Confusion matrix of decision tree

[[12  2  2]
 [ 0 17  0]
 [ 0  1 22]]


In [24]:
print(cmt) #Confusion matrix of svm

[[15  1  0]
 [ 0 17  0]
 [ 0  0 23]]


In [25]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'liblinear')
logisticRegr.fit(train_img,train_label)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
lor=logisticRegr.predict(test_img)
print(lor)

[4. 7. 5. 7. 5. 5. 5. 7. 7. 5. 4. 4. 5. 4. 5. 4. 7. 5. 4. 5. 7. 4. 4. 5.
 5. 4. 7. 7. 7. 7. 5. 4. 4. 7. 5. 5. 5. 4. 7. 5. 7. 5. 7. 4. 7. 5. 4. 7.
 7. 7. 7. 7. 7. 7. 7. 5.]


In [27]:
cm_lor=confusion_matrix(test_label,lor)
a_lor=accuracy_score(test_label,lor)
print(classification_report(test_label,lor))
print("Accuracy of the model with logistic regression classification: \n",a_lor,"\n")
print("Confusion matrix \n",cm_lor)

              precision    recall  f1-score   support

         4.0       1.00      0.88      0.93        16
         5.0       0.89      1.00      0.94        17
         7.0       1.00      1.00      1.00        23

    accuracy                           0.96        56
   macro avg       0.96      0.96      0.96        56
weighted avg       0.97      0.96      0.96        56

Accuracy of the model with logistic regression classification: 
 0.9642857142857143 

Confusion matrix 
 [[14  2  0]
 [ 0 17  0]
 [ 0  0 23]]
